"""
BERT와 GPT
BERT(Bidirectional Encoder Representations from Transformers)
GPT(Generative Pre-trained Transformer)
문서 분류, 질의 응답, 개체명 인식(ner), 문장 생성
EX) 개체명 인식 : 삼성 -> 기업이라고 인식
트랜스퍼러닝(Transfer Learning;전이학습) :pre-trained, fine-tuning
특정 용도에 맞춰 사전에 학습된(upstream  태스크) 모델을 다른 용도로 재사용(downstream 태스크)할 수 있는 기법
업스트림 태스크(언어모델) : 다음 단어 예측(GPT), 빈칸 채우기(BERT)->마스크드 언어 모델, 코퍼스의 문맥을 이해하는 문제->학습->pre-trained(프리트레인드) 모델
다운스트림 태스크 : 문서 분류, 개체명 인식 등 구체적인 자연어 문제
ex) 독도는 우리 땅, 독도는 경상북도에 속한다, ...
EX) 독도는 ____ 땅
eXtrainable AI(설명가능한 AI)
자기지도학습 : 데이터 내에서 X,Y데이터를 나누어 학
파인튜닝 : pre-trained 모델의 가중치를 태스크에 맞춰 모델 전체가 업데이트하는 기법
프롬프트튜닝(prompt tuning) : pre-trained 모델의 가중치를 태스크에 맞춰 모델 일부만 업데이트하는 기법
in-context 러닝 : 다운스트림 테스크 데이터의 일부만 사용, 모델을 업데이트 하지 않음
-퓨샷(few-shot)러닝 : 다운스트림 테스크 데이터 몇 건만 사용
-제로샷(zero-shot)러닝: 다운스트림 테스크 데이터 전혀 사용 안함
-원샷(one-shot)러닝 : : 다운스트림 테스크 데이터 11 건만 사용
토큰화:문자, 단어, 문장, 서브단어
바이트 쌍 인코딩 : 최근 모델 토큰화 기법, GPT
단어 조각 인코딩(워드피스) : BERT
바이트 쌍 인코딩
aaabbcaabb (a,b,c)
xabbcxbb
xaycxy (x, a, y, c)
"""

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/빅리더_자연어처리/kor-eng/kor.txt", sep = '\t', names=['src','tar','lic'])

In [ ]:
del df['lic']

In [ ]:
df

,src,tar
0,Go.,가.
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?
...,...,...
5865,I started a new blog. I'll do my best not to b...,난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되지 ...
5866,I think it's a shame that some foreign languag...,몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 있었...
5867,And the good news is that today the economy is...,"다음으로 좋은 소식은 오늘 경제가 재성장한다는 것입니다. 임금, 소득, 집값, 퇴직..."
5868,If someone who doesn't know your background sa...,만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 원어...


In [ ]:
len(df)

5870

In [ ]:
df.tar=df.tar.apply(lambda x: '\t'+x+'\t')#sos:\t, eos:\n
df
#apply. applymap, filter, lambda, map

,src,tar
0,Go.,\t가.\t
1,Hi.,\t안녕.\t
2,Run!,\t뛰어!\t
3,Run.,\t뛰어.\t
4,Who?,\t누구?\t
...,...,...
5865,I started a new blog. I'll do my best not to b...,\t난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되...
5866,I think it's a shame that some foreign languag...,\t몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 ...
5867,And the good news is that today the economy is...,"\t다음으로 좋은 소식은 오늘 경제가 재성장한다는 것입니다. 임금, 소득, 집값, ..."
5868,If someone who doesn't know your background sa...,\t만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 ...


In [ ]:
srcVocab = set()
for line in df.src:
  for c in line:
    srcVocab.add(c)

In [ ]:
tarVocab=set()
for line in df.tar:
    for c in line:
        tarVocab.add(c)

In [ ]:
len(tarVocab)

1008

In [ ]:
len(srcVocab)

74

In [ ]:
srcVocab=sorted(list(srcVocab))
tarVocab= sorted(list(tarVocab))

In [ ]:
srcVocabSize = len(srcVocab)+1
tarVocabSize = len(tarVocab)+1 #0번은 사용 하지 않음

In [ ]:
tarVocabSize

1009

In [ ]:
src_to_index=dict([(w, i+1) for i, w in enumerate(srcVocab)])
tar_to_index=dict([(w, i+1) for i, w in enumerate(tarVocab)])



In [ ]:
enc_input = []
for line in df.src:
  enc_line=[]
  for c in line:
    enc_line.append(src_to_index[c])
  enc_input.append(enc_line)

In [ ]:
print(enc_input)

[[29, 61, 9], [30, 55, 9], [40, 67, 60, 2], [40, 67, 60, 9], [45, 54, 61, 22], [45, 61, 69, 2], [26, 67, 49, 57, 2], [28, 55, 64, 51, 2], [30, 51, 58, 62, 2], [30, 55, 50, 51, 9], [32, 67, 59, 62, 2], [32, 67, 59, 62, 9], [41, 66, 47, 71, 9], [45, 47, 55, 66, 2], [45, 47, 55, 66, 2], [45, 47, 55, 66, 9], [24, 51, 53, 55, 60, 9], [30, 51, 58, 58, 61, 2], [30, 51, 58, 58, 61, 9], [31, 1, 65, 51, 51, 9], [31, 1, 66, 64, 71, 9], [31, 1, 69, 61, 60, 2], [37, 54, 1, 60, 61, 2], [40, 51, 58, 47, 70, 9], [41, 54, 61, 61, 66, 2], [41, 59, 55, 58, 51, 9], [41, 61, 64, 64, 71, 22], [41, 61, 64, 64, 71, 22], [41, 61, 64, 64, 71, 22], [23, 66, 66, 47, 49, 57, 2], [23, 66, 66, 47, 49, 57, 2], [28, 64, 51, 51, 72, 51, 2], [29, 51, 66, 1, 67, 62, 9], [29, 61, 66, 1, 55, 66, 2], [29, 61, 66, 1, 55, 66, 22], [30, 51, 1, 64, 47, 60, 9], [30, 67, 53, 1, 59, 51, 9], [31, 1, 57, 60, 61, 69, 9], [31, 1, 58, 61, 65, 66, 9], [31, 1, 63, 67, 55, 66, 9], [31, 1, 69, 61, 64, 57, 9], [34, 55, 65, 66, 51, 60, 9], [

In [ ]:
dec_input = []
for line in df.tar:
  dec_line=[]
  for c in line:
    dec_line.append(tar_to_index[c])
  dec_input.append(dec_line)

In [ ]:
print(dec_input)

[[1, 47, 10, 1], [1, 606, 213, 10, 1], [1, 318, 628, 3, 1], [1, 318, 628, 10, 1], [1, 224, 102, 23, 1], [1, 671, 662, 3, 1], [1, 559, 643, 3, 1], [1, 592, 3, 1], [1, 267, 662, 752, 3, 1], [1, 563, 628, 10, 1], [1, 727, 937, 3, 1], [1, 727, 937, 951, 10, 1], [1, 47, 387, 2, 706, 628, 10, 1], [1, 126, 244, 353, 3, 1], [1, 715, 134, 3, 1], [1, 126, 244, 353, 10, 1], [1, 574, 711, 951, 10, 1], [1, 606, 213, 3, 1], [1, 606, 213, 10, 1], [1, 609, 614, 628, 10, 1], [1, 574, 267, 951, 475, 74, 10, 1], [1, 190, 47, 2, 697, 82, 628, 3, 1], [1, 604, 238, 2, 697, 343, 3, 1], [1, 762, 729, 951, 10, 1], [1, 592, 3, 1], [1, 675, 628, 10, 1], [1, 745, 556, 949, 238, 244, 387, 23, 1], [1, 714, 2, 422, 2, 293, 637, 628, 668, 10, 1], [1, 432, 325, 86, 2, 944, 551, 177, 668, 23, 1], [1, 94, 78, 3, 1], [1, 94, 78, 951, 3, 1], [1, 151, 769, 385, 3, 1], [1, 700, 628, 177, 10, 1], [1, 609, 76, 628, 3, 1], [1, 609, 76, 628, 23, 1], [1, 118, 233, 2, 249, 357, 244, 10, 1], [1, 606, 604, 752, 10, 1], [1, 609, 604

In [ ]:
dec_tar=[]
for line in df.tar:
  enc_line =[]
  t=0
  for c in line:
    if t>0:
      enc_line.append(tar_to_index[c])
    t+=1
  dec_tar.append(enc_line)

In [ ]:
dec_tar

[[47, 10, 1],
 [606, 213, 10, 1],
 [318, 628, 3, 1],
 [318, 628, 10, 1],
 [224, 102, 23, 1],
 [671, 662, 3, 1],
 [559, 643, 3, 1],
 [592, 3, 1],
 [267, 662, 752, 3, 1],
 [563, 628, 10, 1],
 [727, 937, 3, 1],
 [727, 937, 951, 10, 1],
 [47, 387, 2, 706, 628, 10, 1],
 [126, 244, 353, 3, 1],
 [715, 134, 3, 1],
 [126, 244, 353, 10, 1],
 [574, 711, 951, 10, 1],
 [606, 213, 3, 1],
 [606, 213, 10, 1],
 [609, 614, 628, 10, 1],
 [574, 267, 951, 475, 74, 10, 1],
 [190, 47, 2, 697, 82, 628, 3, 1],
 [604, 238, 2, 697, 343, 3, 1],
 [762, 729, 951, 10, 1],
 [592, 3, 1],
 [675, 628, 10, 1],
 [745, 556, 949, 238, 244, 387, 23, 1],
 [714, 2, 422, 2, 293, 637, 628, 668, 10, 1],
 [432, 325, 86, 2, 944, 551, 177, 668, 23, 1],
 [94, 78, 3, 1],
 [94, 78, 951, 3, 1],
 [151, 769, 385, 3, 1],
 [700, 628, 177, 10, 1],
 [609, 76, 628, 3, 1],
 [609, 76, 628, 23, 1],
 [118, 233, 2, 249, 357, 244, 10, 1],
 [606, 604, 752, 10, 1],
 [609, 604, 10, 1],
 [177, 233, 2, 702, 628, 457, 357, 244, 10, 1],
 [177, 2, 177, 49, 

In [ ]:
max_src_len=max([len(line) for line in df.src])#영어 문장

In [ ]:
max_tar_len = max([len(line) for line in df.tar])#한국어 문장

In [ ]:
max_tar_len

298

In [ ]:
enc_input=pad_sequences(enc_input, maxlen=max_src_len, padding='pre')
dec_input=pad_sequences(dec_input, maxlen=max_tar_len, padding='pre')
dec_tar=pad_sequences(dec_tar, maxlen=max_tar_len, padding='pre')

In [ ]:
enc_input = to_categorical(enc_input)
dec_input = to_categorical(dec_input)
dec_tar = to_categorical(dec_tar)

In [ ]:
enc_input.shape #개수, 입력으로 올라가는 문장의 길이, 단어에 대한 차원(종류)

(5870, 537, 75)

In [ ]:
dec_input.shape

(5870, 298, 1009)

In [ ]:
dec_tar.shape

(5870, 298, 1009)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
'''
입력차원은 처음에만
model = Sequential()
model.add(Dense(10, input_dim=100, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

2) functional API
Input(shape=(None, srcVocapSize))#입력문장의 개수는 정해져있지 않음. 차원은 단어의 개수
'''

"\n입력차원은 처음에만\nmodel = Sequential()\nmodel.add(Dense(10, input_dim=100, activation='relu'))\nmodel.add(Dense(5, activation='relu'))\nmodel.add(Dense(1, activation='sigmoid'))\n\n2) functional API\nInput(shape=(None, srcVocapSize))#입력문장의 개수는 정해져있지 않음. 차원은 단어의 개수\n"

In [ ]:
#Encoder
enc_inputs = Input(shape=(None, srcVocabSize))
enc_lstm = LSTM(units=256, return_state=True)
enc_outputs, stateH, stateC = enc_lstm(enc_inputs)
enc_states=[stateH, stateC]#context vector


#enc_outputs, stateH, state_C=LSTM(units=256, return_state=True)#return_sequences->셀 출력 제어, return_state -> 인코더의 마지막 셀의 정보를 디코더에 전달하는 작업 제어 (중요)
#return_state해주면 나오는 stateH: 은닉상태정보, state_C: 셀 상태 정보


In [ ]:
#decoder
dec_inputs = Input(shape=(None, tarVocabSize))
dec_lstm = LSTM(units=256, return_state=True, return_sequences=True)

# 인코더 => 정보 => 디코더 (인코더에서 디코더로 정보 전달 받는)
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=enc_states)#initial_state=enc_states가 핵심 코드

dec_layer=Dense(tarVocabSize, activation='softmax')
dec_outputs=dec_layer(dec_outputs)

In [ ]:
model=Model([enc_inputs, dec_inputs], dec_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.fit(x=[enc_input, dec_input], y=dec_tar, batch_size=64, epochs=3,
          validation_split=0.2)

Epoch 1/3
24/74 [========>.....................] - ETA: 1:07 - loss: 1.4496

In [ ]:
enc_input.shape
enc_input[3].shape

(537, 75)

In [ ]:
df.src[3]

'Run.'

In [ ]:
enc_model=(Model(inputs=enc_inputs, outputs = enc_states))

In [ ]:
#상태 정보를 저장하는 변수
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]

In [ ]:
'''
1.입력문장: 영문
2.정답문장: 한글
3.예측번역문장: 한글
'''

In [ ]:
dec_outputs, state_h, state_c=dec_lstm(dec_inputs, initial_state=decoder_states_inputs)

In [ ]:
decoder_states = [state_h, state_c]
dec_outputs = dec_layer(dec_outputs)#softmax수행
decoder_model=Model(inputs=[dec_inputs]+decoder_states_inputs, outputs = [dec_outputs]+decoder_states)

In [ ]:
def dec_sequence(inpiut_seq):


In [ ]:
input_seq = enc_input[3]
dec_sentence=dec_sequence(iuput_seq)